In [366]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
import pandas as pd
from random import random
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
%matplotlib inline

class dummy(ml.classifier):
    def __init__(self,X,Y,P): self.P=P; self.classes=np.unique(Y);
    def predictSoft(self,X): return self.P

In [367]:
X = np.genfromtxt('X_train.txt', delimiter=',')
Y = np.genfromtxt('Y_train.txt', delimiter=',')
Xte = np.genfromtxt('X_test.txt', delimiter=',')

X = X[:,[28,56,7,22,58,24,52,42,5,30,14,47,51,6,1,16,27,94,20,68,
        69,72,85,100,15,0,73,75,76,78,79,83,84,86,87,90,92,96,97,102,33,37,60,61,35]]
Xte = Xte[:,[28,56,7,22,58,24,52,42,5,30,14,47,51,6,1,16,27,94,20,68,
        69,72,85,100,15,0,73,75,76,78,79,83,84,86,87,90,92,96,97,102,33,37,60,61,35]]

print(X.shape)
print(Xte.shape)

[Xtr, Xva, Ytr, Yva] = ml.splitData(X,Y,0.8)

Xtr, param = ml.transforms.rescale(Xtr)
Xva,_ = ml.transforms.rescale(Xva,param)
Xte,_ = ml.transforms.rescale(Xte,param)

(7423, 45)
(7422, 45)


In [368]:
# simple neural net
nHidden = 1000
nnet = ml.nnet.nnetClassify()
nnet.init_weights([Xtr.shape[1],nHidden,2],'random',Xtr,Ytr);
nnet.train(Xtr[:10000,:],Ytr[:10000],stopTol=-1,initStep=0.65,stopIter=150000)

print('Validation AUC: '+ str(nnet.auc(Xva, Yva)))

nn_ytr_hat = nnet.predictSoft(Xtr)
nn_yva_hat = nnet.predictSoft(Xva)
nn_yte_hat = nnet.predictSoft(Xte)

it 2 : Jsur = 0.5000477812449063, J01 = 0.4996631862579993
it 4 : Jsur = 0.4999928090766285, J01 = 0.4976423038059953
it 8 : Jsur = 0.49988780520223935, J01 = 0.4991579656449983
it 16 : Jsur = 0.5031685175572981, J01 = 0.5271135062310542
it 32 : Jsur = 0.5140711395803385, J01 = 0.4922532839339845
it 64 : Jsur = 0.49916522516284195, J01 = 0.4592455372179185
it 128 : Jsur = 0.5105091991986699, J01 = 0.4363422027618727
it 256 : Jsur = 0.5033213610760383, J01 = 0.42842707982485684
it 512 : Jsur = 0.48336859936979576, J01 = 0.42135399124284273
it 1024 : Jsur = 0.48599222122828123, J01 = 0.42286965308184576
it 2048 : Jsur = 0.47091953643147316, J01 = 0.4058605591108117
it 4096 : Jsur = 0.4658735373762376, J01 = 0.39003031323678006
it 8192 : Jsur = 0.4629875641574865, J01 = 0.3912091613337824
it 16384 : Jsur = 0.4618214069471904, J01 = 0.386998989558774
it 32768 : Jsur = 0.4613000747530077, J01 = 0.3878410239137757
it 65536 : Jsur = 0.46110231858436945, J01 = 0.38683058268777365
it 131072 : J

In [370]:
# boosted linear regressors
nUse= 800
mu = Ytr.mean()
step = 0.1
Ft = np.zeros((Xtr.shape[0],)) + np.log(mu/(1-mu))
Fv = np.zeros((Xva.shape[0],)) + np.log(mu/(1-mu))
Fte = np.zeros((Xte.shape[0],)) + np.log(mu/(1-mu))

def sigma(z): return 1./(1.+np.exp(-z))

Pt3 = np.zeros((Xtr.shape[0],2)); Pt3[:,0]=1-mu; Pt3[:,1]=mu;
Pv3 = np.zeros((Xva.shape[0],2)); Pv3[:,0]=1-mu; Pv3[:,1]=mu;
Pte3 = np.zeros((Xte.shape[0],2)); Pte3[:,0]=1-mu; Pte3[:,1]=mu;

np.random.seed(0)
for l in range(nUse): # this is a lot faster than the bagging loop:
    dJ = 1.*Ytr - sigma(Ft)
    #tree = ml.dtree.treeRegress(Xtr,dJ, maxDepth=4) # train and save predictions
    linc = ml.linear.linearRegress(Xtr,dJ,5.2)
    
    Ft += step*np.reshape(linc.predict(Xtr), (linc.predict(Xtr).shape[0],))
    Fv += step*np.reshape(linc.predict(Xva), (linc.predict(Xva).shape[0],))
    Fte += step*np.reshape(linc.predict(Xte), (linc.predict(Xte).shape[0],))
    
    Pt3[:,1] = sigma(Ft); Pt3[:,0] = 1-Pt3[:,1]
    Pv3[:,1] = sigma(Fv); Pv3[:,0] = 1-Pv3[:,1]
    Pte3[:,1] = sigma(Fte); Pte3[:,0] = 1-Pte3[:,1]
    
lin_ytr_hat = Pt3
lin_yva_hat = Pv3
lin_yte_hat = Pte3
    
print('Validation AUC: ' + str(dummy(Xva,Yva,lin_yva_hat).auc(Xva,Yva)))    
print('Validation Error: ', np.mean((Yva - lin_yva_hat[:,1])**2))

Validation AUC: 0.6731608569353668
Validation Error:  0.22934751895865996


In [371]:
# boosted
nUse=128
mu = Ytr.mean()
step = 0.1
Ft = np.zeros((Xtr.shape[0],)) + np.log(mu/(1-mu))
Fv = np.zeros((Xva.shape[0],)) + np.log(mu/(1-mu))
Fte = np.zeros((Xte.shape[0],)) + np.log(mu/(1-mu))

def sigma(z): return 1./(1.+np.exp(-z))

Pt2 = np.zeros((Xtr.shape[0],2)); Pt2[:,0]=1-mu; Pt2[:,1]=mu;
Pv2 = np.zeros((Xva.shape[0],2)); Pv2[:,0]=1-mu; Pv2[:,1]=mu;
Pte2 = np.zeros((Xte.shape[0],2)); Pte2[:,0]=1-mu; Pte2[:,1]=mu;

np.random.seed(0)
for l in range(nUse): # this is a lot faster than the bagging loop:
    dJ = 1.*Ytr - sigma(Ft)
    tree = ml.dtree.treeRegress(Xtr,dJ, maxDepth=3) # train and save predictions
    Ft += step*tree.predict(Xtr)
    Fv += step*tree.predict(Xva)
    Fte += step*tree.predict(Xte)
    
    Pt2[:,1] = sigma(Ft); Pt2[:,0] = 1-Pt2[:,1]
    Pv2[:,1] = sigma(Fv); Pv2[:,0] = 1-Pv2[:,1]
    Pte2[:,1] = sigma(Fte); Pte2[:,0] = 1-Pte2[:,1]
    
grad_tree_ytr_hat = Pt2
grad_tree_yva_hat = Pv2
grad_tree_yte_hat = Pte2

In [372]:
print('Validation AUC: ' + str(dummy(Xva,Yva,grad_tree_yva_hat).auc(Xva,Yva)))
print('Validation Error: ' + str(np.mean((Yva - grad_tree_yva_hat[:,1])**2)))

Validation AUC: 0.7511283587509077
Validation Error: 0.20306751059209563


In [373]:
# bagged forest
ensemble = []
forest_yva_hat = np.ones((Xva.shape[0], 2))
forest_ytr_hat = np.ones((Xtr.shape[0], 2))
forest_yte_hat = np.ones((Xte.shape[0], 2))

nUse = 128
for i in range(nUse):
    x_boot, y_boot = ml.bootstrapData(Xtr, Ytr, n_boot = Xtr.shape[0])
    ensemble.append(ml.dtree.treeClassify(x_boot, y_boot, minParent=64, maxDepth=10, minLeaf=4, nFeatures=13))
    
    forest_yva_hat += ensemble[i].predictSoft(Xva)
    forest_ytr_hat += ensemble[i].predictSoft(Xtr)
    forest_yte_hat += ensemble[i].predictSoft(Xte)
    
forest_yva_hat /= nUse
forest_ytr_hat /= nUse
forest_yte_hat /= nUse

In [374]:
print('Validation AUC: ' + str(dummy(Xva,Yva,forest_yva_hat).auc(Xva,Yva)))
print('Validation Error: ' + str(np.mean((Yva - forest_yva_hat[:,1])**2)))

Validation AUC: 0.7512835875090778
Validation Error: 0.20001326778038503


In [378]:
Sva = np.hstack((nn_yva_hat, lin_yva_hat, grad_tree_yva_hat, forest_yva_hat))
stack = ml.linearC.linearClassify(Sva, Yva, reg=1e-20)
print('Stacked AUC: ' + str(stack.auc(Sva,Yva)))

Stacked AUC: 0.7537182280319535


In [377]:
Str = np.hstack((nn_ytr_hat, lin_ytr_hat, grad_tree_ytr_hat, forest_ytr_hat))
ytr_hat = stack.predictSoft(Str)
yva_hat = stack.predictSoft(Sva)

print('Stacked Training Error: ' + str(np.mean((Ytr - ytr_hat[:,1])**2)))
print('Stacked Validation Error: ' + str(np.mean((Yva - yva_hat[:,1])**2)))

Stacked Training Error: 0.17178736435999392
Stacked Validation Error: 0.1978226756149664


In [379]:
Ste = np.hstack((nn_yte_hat, lin_yte_hat, grad_tree_yte_hat, forest_yte_hat))
final_ste_hat = stack.predictSoft(Ste)

In [380]:
print(final_ste_hat[:,1].min())
print(final_ste_hat[:,1].max())

0.055103571752166354
0.9699543706225553


In [381]:
result = []
for i, r in enumerate(final_ste_hat[:,1]):
    result.append([i,r])

In [382]:
#final = np.vstack((np.arange(final_y_hat.shape[0]), final_y_hat[:,1])).T
np.savetxt('projectresults.txt', result,'%d, %.2f',header='Id,Predicted',comments='', delimiter=',')